In [11]:
pip install requests beautifulsoup4 pandas


Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install selenium



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#2024 final 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Setup headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    return rating_tag.get_text(strip=True) if rating_tag else 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        fraction = soup.select_one("span.a-price-fraction")
        return price_span.get_text(strip=True) + "." + fraction.get_text(strip=True) if fraction else price_span.get_text(strip=True)
    return 'N/A'

def extract_reviews(soup):
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    return reviews_tag.get_text(strip=True) if reviews_tag else 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()
        if data['docs']:
            book_info = data['docs'][0]
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception:
        return 'Unknown', 'Unknown'

def extract_year(soup):
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "Publisher" in text:
            match = re.search(r"(\d{4})", text)
            if match:
                return match.group(1)
    return 'Unknown'

def extract_pages(soup):
    pages = 'Unknown'
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "pages" in text.lower():
            pages_match = re.search(r"(\d+)\s+pages", text)
            if pages_match:
                pages = pages_match.group(1)
    return pages

def scrape_amazon_bestsellers(year):
    print(f"Scraping Amazon Bestsellers for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 6):  # Pages 1–5 = 100 books
        url = f"{base_url}?pg={page}"
        print(f"Fetching page {page}...")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title_tag = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
                author_tag = card.select_one("div.a-row.a-size-small")
                link_tag = card.select_one("a.a-link-normal")

                if not (title_tag and link_tag and author_tag):
                    continue

                title = title_tag.get_text(strip=True)
                author = author_tag.get_text(strip=True)
                book_url = "https://www.amazon.com" + link_tag['href']

                # Go to book detail page
                driver.get(book_url)
                time.sleep(1.5)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')

                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)
                reviews = extract_reviews(book_soup)
                price = extract_price(book_soup)

                publisher, published_year = fetch_book_info_from_openlibrary(title)
                amazon_year = extract_year(book_soup)
                language, pages = extract_language_pages(book_soup)

                if published_year == 'Unknown':
                    published_year = amazon_year

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'pages': pages,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping book: {e}")
                continue

    return book_data

# === Set the year to scrape ===
year = 2024
books = scrape_amazon_bestsellers(year)

# Save results to CSV
df = pd.DataFrame(books)
df.to_csv(f"amazon_bestsellers_{year}.csv", index=False)
driver.quit()
print(f"Saved {len(df)} books to amazon_bestsellers_{year}.csv")


Scraping Amazon Bestsellers for 2024...
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Saved 60 books to amazon_bestsellers_2024.csv


In [161]:
#2025 final 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Setup headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    return rating_tag.get_text(strip=True) if rating_tag else 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        fraction = soup.select_one("span.a-price-fraction")
        return price_span.get_text(strip=True) + "." + fraction.get_text(strip=True) if fraction else price_span.get_text(strip=True)
    return 'N/A'

def extract_reviews(soup):
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    return reviews_tag.get_text(strip=True) if reviews_tag else 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()
        if data['docs']:
            book_info = data['docs'][0]
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception:
        return 'Unknown', 'Unknown'

def extract_year(soup):
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "Publisher" in text:
            match = re.search(r"(\d{4})", text)
            if match:
                return match.group(1)
    return 'Unknown'

def extract_pages(soup):
    pages = 'Unknown'
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "pages" in text.lower():
            pages_match = re.search(r"(\d+)\s+pages", text)
            if pages_match:
                pages = pages_match.group(1)
    return pages

def scrape_amazon_bestsellers(year):
    print(f"Scraping Amazon Bestsellers for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 6):  # Pages 1–5 = 100 books
        url = f"{base_url}?pg={page}"
        print(f"Fetching page {page}...")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title_tag = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
                author_tag = card.select_one("div.a-row.a-size-small")
                link_tag = card.select_one("a.a-link-normal")

                if not (title_tag and link_tag and author_tag):
                    continue

                title = title_tag.get_text(strip=True)
                author = author_tag.get_text(strip=True)
                book_url = "https://www.amazon.com" + link_tag['href']

                # Go to book detail page
                driver.get(book_url)
                time.sleep(1.5)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')

                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)
                reviews = extract_reviews(book_soup)
                price = extract_price(book_soup)

                publisher, published_year = fetch_book_info_from_openlibrary(title)
                amazon_year = extract_year(book_soup)
                language, pages = extract_language_pages(book_soup)

                if published_year == 'Unknown':
                    published_year = amazon_year

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'pages': pages,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping book: {e}")
                continue

    return book_data

# === Set the year to scrape ===
year = 2025
books = scrape_amazon_bestsellers(year)

# Save results to CSV
df = pd.DataFrame(books)
df.to_csv(f"amazon_bestsellers_{year}.csv", index=False)
driver.quit()
print(f"Saved {len(df)} books to amazon_bestsellers_{year}.csv")

Scraping Amazon Bestsellers for 2025...
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Saved 60 books to amazon_bestsellers_2025.csv


In [162]:
#2023 final 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Setup headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    return rating_tag.get_text(strip=True) if rating_tag else 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        fraction = soup.select_one("span.a-price-fraction")
        return price_span.get_text(strip=True) + "." + fraction.get_text(strip=True) if fraction else price_span.get_text(strip=True)
    return 'N/A'

def extract_reviews(soup):
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    return reviews_tag.get_text(strip=True) if reviews_tag else 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()
        if data['docs']:
            book_info = data['docs'][0]
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception:
        return 'Unknown', 'Unknown'

def extract_year(soup):
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "Publisher" in text:
            match = re.search(r"(\d{4})", text)
            if match:
                return match.group(1)
    return 'Unknown'

def extract_pages(soup):
    pages = 'Unknown'
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "pages" in text.lower():
            pages_match = re.search(r"(\d+)\s+pages", text)
            if pages_match:
                pages = pages_match.group(1)
    return pages

def scrape_amazon_bestsellers(year):
    print(f"Scraping Amazon Bestsellers for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 6):  # Pages 1–5 = 100 books
        url = f"{base_url}?pg={page}"
        print(f"Fetching page {page}...")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title_tag = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
                author_tag = card.select_one("div.a-row.a-size-small")
                link_tag = card.select_one("a.a-link-normal")

                if not (title_tag and link_tag and author_tag):
                    continue

                title = title_tag.get_text(strip=True)
                author = author_tag.get_text(strip=True)
                book_url = "https://www.amazon.com" + link_tag['href']

                # Go to book detail page
                driver.get(book_url)
                time.sleep(1.5)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')

                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)
                reviews = extract_reviews(book_soup)
                price = extract_price(book_soup)

                publisher, published_year = fetch_book_info_from_openlibrary(title)
                amazon_year = extract_year(book_soup)
                language, pages = extract_language_pages(book_soup)

                if published_year == 'Unknown':
                    published_year = amazon_year

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'pages': pages,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping book: {e}")
                continue

    return book_data

# === Set the year to scrape ===
year = 2023
books = scrape_amazon_bestsellers(year)

# Save results to CSV
df = pd.DataFrame(books)
df.to_csv(f"amazon_bestsellers_{year}.csv", index=False)
driver.quit()
print(f"Saved {len(df)} books to amazon_bestsellers_{year}.csv")

Scraping Amazon Bestsellers for 2023...
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Saved 60 books to amazon_bestsellers_2023.csv


In [163]:
#2022 final 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Setup headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    return rating_tag.get_text(strip=True) if rating_tag else 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        fraction = soup.select_one("span.a-price-fraction")
        return price_span.get_text(strip=True) + "." + fraction.get_text(strip=True) if fraction else price_span.get_text(strip=True)
    return 'N/A'

def extract_reviews(soup):
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    return reviews_tag.get_text(strip=True) if reviews_tag else 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()
        if data['docs']:
            book_info = data['docs'][0]
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception:
        return 'Unknown', 'Unknown'

def extract_year(soup):
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "Publisher" in text:
            match = re.search(r"(\d{4})", text)
            if match:
                return match.group(1)
    return 'Unknown'

def extract_pages(soup):
    pages = 'Unknown'
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "pages" in text.lower():
            pages_match = re.search(r"(\d+)\s+pages", text)
            if pages_match:
                pages = pages_match.group(1)
    return pages

def scrape_amazon_bestsellers(year):
    print(f"Scraping Amazon Bestsellers for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 6):  # Pages 1–5 = 100 books
        url = f"{base_url}?pg={page}"
        print(f"Fetching page {page}...")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title_tag = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
                author_tag = card.select_one("div.a-row.a-size-small")
                link_tag = card.select_one("a.a-link-normal")

                if not (title_tag and link_tag and author_tag):
                    continue

                title = title_tag.get_text(strip=True)
                author = author_tag.get_text(strip=True)
                book_url = "https://www.amazon.com" + link_tag['href']

                # Go to book detail page
                driver.get(book_url)
                time.sleep(1.5)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')

                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)
                reviews = extract_reviews(book_soup)
                price = extract_price(book_soup)

                publisher, published_year = fetch_book_info_from_openlibrary(title)
                amazon_year = extract_year(book_soup)
                language, pages = extract_language_pages(book_soup)

                if published_year == 'Unknown':
                    published_year = amazon_year

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'pages': pages,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping book: {e}")
                continue

    return book_data

# === Set the year to scrape ===
year = 2022
books = scrape_amazon_bestsellers(year)

# Save results to CSV
df = pd.DataFrame(books)
df.to_csv(f"amazon_bestsellers_{year}.csv", index=False)
driver.quit()
print(f"Saved {len(df)} books to amazon_bestsellers_{year}.csv")

Scraping Amazon Bestsellers for 2022...
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Saved 60 books to amazon_bestsellers_2022.csv


In [164]:
#2021 final 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Setup headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    return rating_tag.get_text(strip=True) if rating_tag else 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        fraction = soup.select_one("span.a-price-fraction")
        return price_span.get_text(strip=True) + "." + fraction.get_text(strip=True) if fraction else price_span.get_text(strip=True)
    return 'N/A'

def extract_reviews(soup):
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    return reviews_tag.get_text(strip=True) if reviews_tag else 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()
        if data['docs']:
            book_info = data['docs'][0]
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception:
        return 'Unknown', 'Unknown'

def extract_year(soup):
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "Publisher" in text:
            match = re.search(r"(\d{4})", text)
            if match:
                return match.group(1)
    return 'Unknown'

def extract_pages(soup):
    pages = 'Unknown'
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "pages" in text.lower():
            pages_match = re.search(r"(\d+)\s+pages", text)
            if pages_match:
                pages = pages_match.group(1)
    return pages

def scrape_amazon_bestsellers(year):
    print(f"Scraping Amazon Bestsellers for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 6):  # Pages 1–5 = 100 books
        url = f"{base_url}?pg={page}"
        print(f"Fetching page {page}...")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title_tag = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
                author_tag = card.select_one("div.a-row.a-size-small")
                link_tag = card.select_one("a.a-link-normal")

                if not (title_tag and link_tag and author_tag):
                    continue

                title = title_tag.get_text(strip=True)
                author = author_tag.get_text(strip=True)
                book_url = "https://www.amazon.com" + link_tag['href']

                # Go to book detail page
                driver.get(book_url)
                time.sleep(1.5)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')

                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)
                reviews = extract_reviews(book_soup)
                price = extract_price(book_soup)

                publisher, published_year = fetch_book_info_from_openlibrary(title)
                amazon_year = extract_year(book_soup)
                language, pages = extract_language_pages(book_soup)

                if published_year == 'Unknown':
                    published_year = amazon_year

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'pages': pages,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping book: {e}")
                continue

    return book_data

# === Set the year to scrape ===
year = 2021
books = scrape_amazon_bestsellers(year)

# Save results to CSV
df = pd.DataFrame(books)
df.to_csv(f"amazon_bestsellers_{year}.csv", index=False)
driver.quit()
print(f"Saved {len(df)} books to amazon_bestsellers_{year}.csv")

Scraping Amazon Bestsellers for 2021...
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Saved 60 books to amazon_bestsellers_2021.csv


In [165]:
#2020 final 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Setup headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    return rating_tag.get_text(strip=True) if rating_tag else 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        fraction = soup.select_one("span.a-price-fraction")
        return price_span.get_text(strip=True) + "." + fraction.get_text(strip=True) if fraction else price_span.get_text(strip=True)
    return 'N/A'

def extract_reviews(soup):
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    return reviews_tag.get_text(strip=True) if reviews_tag else 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()
        if data['docs']:
            book_info = data['docs'][0]
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception:
        return 'Unknown', 'Unknown'

def extract_year(soup):
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "Publisher" in text:
            match = re.search(r"(\d{4})", text)
            if match:
                return match.group(1)
    return 'Unknown'

def extract_pages(soup):
    pages = 'Unknown'
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "pages" in text.lower():
            pages_match = re.search(r"(\d+)\s+pages", text)
            if pages_match:
                pages = pages_match.group(1)
    return pages

def scrape_amazon_bestsellers(year):
    print(f"Scraping Amazon Bestsellers for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 6):  # Pages 1–5 = 100 books
        url = f"{base_url}?pg={page}"
        print(f"Fetching page {page}...")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title_tag = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
                author_tag = card.select_one("div.a-row.a-size-small")
                link_tag = card.select_one("a.a-link-normal")

                if not (title_tag and link_tag and author_tag):
                    continue

                title = title_tag.get_text(strip=True)
                author = author_tag.get_text(strip=True)
                book_url = "https://www.amazon.com" + link_tag['href']

                # Go to book detail page
                driver.get(book_url)
                time.sleep(1.5)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')

                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)
                reviews = extract_reviews(book_soup)
                price = extract_price(book_soup)

                publisher, published_year = fetch_book_info_from_openlibrary(title)
                amazon_year = extract_year(book_soup)
                language, pages = extract_language_pages(book_soup)

                if published_year == 'Unknown':
                    published_year = amazon_year

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'pages': pages,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping book: {e}")
                continue

    return book_data

# === Set the year to scrape ===
year = 2020
books = scrape_amazon_bestsellers(year)

# Save results to CSV
df = pd.DataFrame(books)
df.to_csv(f"amazon_bestsellers_{year}.csv", index=False)
driver.quit()
print(f"Saved {len(df)} books to amazon_bestsellers_{year}.csv")

Scraping Amazon Bestsellers for 2020...
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Saved 60 books to amazon_bestsellers_2020.csv


In [166]:
#2019 final 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Setup headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    return rating_tag.get_text(strip=True) if rating_tag else 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        fraction = soup.select_one("span.a-price-fraction")
        return price_span.get_text(strip=True) + "." + fraction.get_text(strip=True) if fraction else price_span.get_text(strip=True)
    return 'N/A'

def extract_reviews(soup):
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    return reviews_tag.get_text(strip=True) if reviews_tag else 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()
        if data['docs']:
            book_info = data['docs'][0]
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception:
        return 'Unknown', 'Unknown'

def extract_year(soup):
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "Publisher" in text:
            match = re.search(r"(\d{4})", text)
            if match:
                return match.group(1)
    return 'Unknown'

def extract_pages(soup):
    pages = 'Unknown'
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "pages" in text.lower():
            pages_match = re.search(r"(\d+)\s+pages", text)
            if pages_match:
                pages = pages_match.group(1)
    return pages

def scrape_amazon_bestsellers(year):
    print(f"Scraping Amazon Bestsellers for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 6):  # Pages 1–5 = 100 books
        url = f"{base_url}?pg={page}"
        print(f"Fetching page {page}...")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title_tag = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
                author_tag = card.select_one("div.a-row.a-size-small")
                link_tag = card.select_one("a.a-link-normal")

                if not (title_tag and link_tag and author_tag):
                    continue

                title = title_tag.get_text(strip=True)
                author = author_tag.get_text(strip=True)
                book_url = "https://www.amazon.com" + link_tag['href']

                # Go to book detail page
                driver.get(book_url)
                time.sleep(1.5)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')

                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)
                reviews = extract_reviews(book_soup)
                price = extract_price(book_soup)

                publisher, published_year = fetch_book_info_from_openlibrary(title)
                amazon_year = extract_year(book_soup)
                language, pages = extract_language_pages(book_soup)

                if published_year == 'Unknown':
                    published_year = amazon_year

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'pages': pages,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping book: {e}")
                continue

    return book_data

# === Set the year to scrape ===
year = 2019
books = scrape_amazon_bestsellers(year)

# Save results to CSV
df = pd.DataFrame(books)
df.to_csv(f"amazon_bestsellers_{year}.csv", index=False)
driver.quit()
print(f"Saved {len(df)} books to amazon_bestsellers_{year}.csv")

Scraping Amazon Bestsellers for 2019...
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Saved 59 books to amazon_bestsellers_2019.csv


In [167]:
#2018 final 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Setup headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    return rating_tag.get_text(strip=True) if rating_tag else 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        fraction = soup.select_one("span.a-price-fraction")
        return price_span.get_text(strip=True) + "." + fraction.get_text(strip=True) if fraction else price_span.get_text(strip=True)
    return 'N/A'

def extract_reviews(soup):
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    return reviews_tag.get_text(strip=True) if reviews_tag else 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()
        if data['docs']:
            book_info = data['docs'][0]
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception:
        return 'Unknown', 'Unknown'

def extract_year(soup):
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "Publisher" in text:
            match = re.search(r"(\d{4})", text)
            if match:
                return match.group(1)
    return 'Unknown'

def extract_pages(soup):
    pages = 'Unknown'
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "pages" in text.lower():
            pages_match = re.search(r"(\d+)\s+pages", text)
            if pages_match:
                pages = pages_match.group(1)
    return pages

def scrape_amazon_bestsellers(year):
    print(f"Scraping Amazon Bestsellers for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 6):  # Pages 1–5 = 100 books
        url = f"{base_url}?pg={page}"
        print(f"Fetching page {page}...")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title_tag = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
                author_tag = card.select_one("div.a-row.a-size-small")
                link_tag = card.select_one("a.a-link-normal")

                if not (title_tag and link_tag and author_tag):
                    continue

                title = title_tag.get_text(strip=True)
                author = author_tag.get_text(strip=True)
                book_url = "https://www.amazon.com" + link_tag['href']

                # Go to book detail page
                driver.get(book_url)
                time.sleep(1.5)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')

                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)
                reviews = extract_reviews(book_soup)
                price = extract_price(book_soup)

                publisher, published_year = fetch_book_info_from_openlibrary(title)
                amazon_year = extract_year(book_soup)
                language, pages = extract_language_pages(book_soup)

                if published_year == 'Unknown':
                    published_year = amazon_year

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'pages': pages,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping book: {e}")
                continue

    return book_data

# === Set the year to scrape ===
year = 2018
books = scrape_amazon_bestsellers(year)

# Save results to CSV
df = pd.DataFrame(books)
df.to_csv(f"amazon_bestsellers_{year}.csv", index=False)
driver.quit()
print(f"Saved {len(df)} books to amazon_bestsellers_{year}.csv")

Scraping Amazon Bestsellers for 2018...
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Saved 58 books to amazon_bestsellers_2018.csv


In [168]:
#2017 final 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Setup headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    return rating_tag.get_text(strip=True) if rating_tag else 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        fraction = soup.select_one("span.a-price-fraction")
        return price_span.get_text(strip=True) + "." + fraction.get_text(strip=True) if fraction else price_span.get_text(strip=True)
    return 'N/A'

def extract_reviews(soup):
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    return reviews_tag.get_text(strip=True) if reviews_tag else 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()
        if data['docs']:
            book_info = data['docs'][0]
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception:
        return 'Unknown', 'Unknown'

def extract_year(soup):
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "Publisher" in text:
            match = re.search(r"(\d{4})", text)
            if match:
                return match.group(1)
    return 'Unknown'

def extract_pages(soup):
    pages = 'Unknown'
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "pages" in text.lower():
            pages_match = re.search(r"(\d+)\s+pages", text)
            if pages_match:
                pages = pages_match.group(1)
    return pages

def scrape_amazon_bestsellers(year):
    print(f"Scraping Amazon Bestsellers for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 6):  # Pages 1–5 = 100 books
        url = f"{base_url}?pg={page}"
        print(f"Fetching page {page}...")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title_tag = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
                author_tag = card.select_one("div.a-row.a-size-small")
                link_tag = card.select_one("a.a-link-normal")

                if not (title_tag and link_tag and author_tag):
                    continue

                title = title_tag.get_text(strip=True)
                author = author_tag.get_text(strip=True)
                book_url = "https://www.amazon.com" + link_tag['href']

                # Go to book detail page
                driver.get(book_url)
                time.sleep(1.5)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')

                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)
                reviews = extract_reviews(book_soup)
                price = extract_price(book_soup)

                publisher, published_year = fetch_book_info_from_openlibrary(title)
                amazon_year = extract_year(book_soup)
                language, pages = extract_language_pages(book_soup)

                if published_year == 'Unknown':
                    published_year = amazon_year

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'pages': pages,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping book: {e}")
                continue

    return book_data

# === Set the year to scrape ===
year = 2017
books = scrape_amazon_bestsellers(year)

# Save results to CSV
df = pd.DataFrame(books)
df.to_csv(f"amazon_bestsellers_{year}.csv", index=False)
driver.quit()
print(f"Saved {len(df)} books to amazon_bestsellers_{year}.csv")

Scraping Amazon Bestsellers for 2017...
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Saved 59 books to amazon_bestsellers_2017.csv


In [169]:
#2016 final 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Setup headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    return rating_tag.get_text(strip=True) if rating_tag else 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        fraction = soup.select_one("span.a-price-fraction")
        return price_span.get_text(strip=True) + "." + fraction.get_text(strip=True) if fraction else price_span.get_text(strip=True)
    return 'N/A'

def extract_reviews(soup):
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    return reviews_tag.get_text(strip=True) if reviews_tag else 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()
        if data['docs']:
            book_info = data['docs'][0]
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception:
        return 'Unknown', 'Unknown'

def extract_year(soup):
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "Publisher" in text:
            match = re.search(r"(\d{4})", text)
            if match:
                return match.group(1)
    return 'Unknown'

def extract_pages(soup):
    pages = 'Unknown'
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "pages" in text.lower():
            pages_match = re.search(r"(\d+)\s+pages", text)
            if pages_match:
                pages = pages_match.group(1)
    return pages

def scrape_amazon_bestsellers(year):
    print(f"Scraping Amazon Bestsellers for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 6):  # Pages 1–5 = 100 books
        url = f"{base_url}?pg={page}"
        print(f"Fetching page {page}...")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title_tag = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
                author_tag = card.select_one("div.a-row.a-size-small")
                link_tag = card.select_one("a.a-link-normal")

                if not (title_tag and link_tag and author_tag):
                    continue

                title = title_tag.get_text(strip=True)
                author = author_tag.get_text(strip=True)
                book_url = "https://www.amazon.com" + link_tag['href']

                # Go to book detail page
                driver.get(book_url)
                time.sleep(1.5)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')

                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)
                reviews = extract_reviews(book_soup)
                price = extract_price(book_soup)

                publisher, published_year = fetch_book_info_from_openlibrary(title)
                amazon_year = extract_year(book_soup)
                language, pages = extract_language_pages(book_soup)

                if published_year == 'Unknown':
                    published_year = amazon_year

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'pages': pages,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping book: {e}")
                continue

    return book_data

# === Set the year to scrape ===
year = 2016
books = scrape_amazon_bestsellers(year)

# Save results to CSV
df = pd.DataFrame(books)
df.to_csv(f"amazon_bestsellers_{year}.csv", index=False)
driver.quit()
print(f"Saved {len(df)} books to amazon_bestsellers_{year}.csv")

Scraping Amazon Bestsellers for 2016...
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Saved 60 books to amazon_bestsellers_2016.csv


In [170]:
#2015 final 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re

# Setup headless browser
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def extract_genre(soup):
    breadcrumb = soup.select_one("#wayfinding-breadcrumbs_feature_div")
    if breadcrumb:
        genre_links = breadcrumb.find_all("a")
        genres = [g.get_text(strip=True) for g in genre_links if g.get_text(strip=True)]
        return genres[-1] if genres else 'Unknown'
    return 'Unknown'

def extract_rating(soup):
    rating_tag = soup.select_one("span.a-declarative a.a-popover-trigger span.a-size-base.a-color-base")
    return rating_tag.get_text(strip=True) if rating_tag else 'N/A'

def extract_price(soup):
    price_span = soup.select_one("span.a-price span.a-price-whole")
    if price_span:
        fraction = soup.select_one("span.a-price-fraction")
        return price_span.get_text(strip=True) + "." + fraction.get_text(strip=True) if fraction else price_span.get_text(strip=True)
    return 'N/A'

def extract_reviews(soup):
    reviews_tag = soup.select_one("span#acrCustomerReviewText")
    return reviews_tag.get_text(strip=True) if reviews_tag else 'N/A'

def fetch_book_info_from_openlibrary(title):
    try:
        url = f"https://openlibrary.org/search.json?title={title}"
        response = requests.get(url)
        data = response.json()
        if data['docs']:
            book_info = data['docs'][0]
            publisher = book_info.get('publisher', ['Unknown'])[0]
            published_year = book_info.get('first_publish_year', 'Unknown')
            return publisher, published_year
        else:
            return 'Unknown', 'Unknown'
    except Exception:
        return 'Unknown', 'Unknown'

def extract_year(soup):
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "Publisher" in text:
            match = re.search(r"(\d{4})", text)
            if match:
                return match.group(1)
    return 'Unknown'

def extract_pages(soup):
    pages = 'Unknown'
    details = soup.select("div#detailBullets_feature_div span.a-list-item")
    for item in details:
        text = item.get_text(strip=True)
        if "pages" in text.lower():
            pages_match = re.search(r"(\d+)\s+pages", text)
            if pages_match:
                pages = pages_match.group(1)
    return pages

def scrape_amazon_bestsellers(year):
    print(f"Scraping Amazon Bestsellers for {year}...")
    base_url = f"https://www.amazon.com/gp/bestsellers/{year}/books"
    book_data = []

    for page in range(1, 6):  # Pages 1–5 = 100 books
        url = f"{base_url}?pg={page}"
        print(f"Fetching page {page}...")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        book_cards = soup.select("div.zg-grid-general-faceout")

        for card in book_cards:
            try:
                title_tag = card.select_one("div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
                author_tag = card.select_one("div.a-row.a-size-small")
                link_tag = card.select_one("a.a-link-normal")

                if not (title_tag and link_tag and author_tag):
                    continue

                title = title_tag.get_text(strip=True)
                author = author_tag.get_text(strip=True)
                book_url = "https://www.amazon.com" + link_tag['href']

                # Go to book detail page
                driver.get(book_url)
                time.sleep(1.5)
                book_soup = BeautifulSoup(driver.page_source, 'html.parser')

                genre = extract_genre(book_soup)
                rating = extract_rating(book_soup)
                reviews = extract_reviews(book_soup)
                price = extract_price(book_soup)

                publisher, published_year = fetch_book_info_from_openlibrary(title)
                amazon_year = extract_year(book_soup)
                language, pages = extract_language_pages(book_soup)

                if published_year == 'Unknown':
                    published_year = amazon_year

                book_data.append({
                    'title': title,
                    'author': author,
                    'genre': genre,
                    'rating': rating,
                    'price': price,
                    'year': published_year,
                    'pages': pages,
                    'reviews': reviews,
                    'amazon_year': year
                })

            except Exception as e:
                print(f"Error scraping book: {e}")
                continue

    return book_data

# === Set the year to scrape ===
year = 2015
books = scrape_amazon_bestsellers(year)

# Save results to CSV
df = pd.DataFrame(books)
df.to_csv(f"amazon_bestsellers_{year}.csv", index=False)
driver.quit()
print(f"Saved {len(df)} books to amazon_bestsellers_{year}.csv")

Scraping Amazon Bestsellers for 2015...
Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Saved 60 books to amazon_bestsellers_2015.csv


In [ ]:
# Combining all csv
import pandas as pd
import glob

# Load all CSV files from 2015 to 2025
years = range(2015, 2026)
file_names = [f"amazon_bestsellers_{year}.csv" for year in years]

# Read and combine all dataframes
df_list = [pd.read_csv(file) for file in file_names]
combined_df = pd.concat(df_list, ignore_index=True)

# Save to a new CSV
combined_df.to_csv("amazon_bestsellers_2015_2025.csv", index=False)

print("Combined CSV saved as amazon_bestsellers_2015_2025.csv")

Combined CSV saved as amazon_bestsellers_2015_2025.csv


In [ ]:
#Fit all page count
import pandas as pd
import requests
import time

# Load your dataset
df = pd.read_csv("amazon_bestsellers_2015_2025.csv")

# Clean column name if needed
df.columns = [col.strip().lower() for col in df.columns]

# Function to get page count from Open Library
def get_pages_from_openlibrary(title, author):
    query = f"{title} {author}"
    url = f"https://openlibrary.org/search.json?title={title}&author={author}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data["docs"]:
            for book in data["docs"]:
                pages = book.get("number_of_pages_median")
                if pages:
                    return pages
    except:
        pass
    return None

# Replace 'Unknown' in 'pages' column
for idx, row in df.iterrows():
    if str(row["pages"]).strip().lower() == "unknown":
        title, author = row["title"], row["author"]
        print(f"Looking up pages for: {title} by {author}...")
        pages = get_pages_from_openlibrary(title, author)
        if pages:
            df.at[idx, "pages"] = pages
        time.sleep(0.5)  # Be polite to the API


# Save updated CSV
df.to_csv("amazon_bestsellers_pages_filled.csv", index=False)
print("✅ Updated dataset saved as 'amazon_bestsellers_pages_filled.csv'")


Looking up pages for: D&D 2014 Player's Handbook (Dungeons & Dragons Core Rulebook) by Dungeons & Dragons...
Looking up pages for: Scholastic Flash Cards: Sight Words by Scholastic Teacher Resources...
Looking up pages for: Harry Potter Paperback Box Set (Books 1-7) by J. K. Rowling...
Looking up pages for: Harry Potter Paperback Box Set (Books 1-7) by J. K. Rowling...
Looking up pages for: Scholastic Flash Cards: Sight Words by Scholastic Teacher Resources...
Looking up pages for: D&D 2014 Player's Handbook (Dungeons & Dragons Core Rulebook) by Dungeons & Dragons...
Looking up pages for: Scholastic Flash Cards: Sight Words by Scholastic Teacher Resources...
Looking up pages for: Scholastic Flash Cards: Sight Words by Scholastic Teacher Resources...
✅ Updated dataset saved as 'amazon_bestsellers_pages_filled.csv'


In [179]:
df = pd.read_csv("amazon_bestsellers_pages_filled.csv")

In [ ]:
df.isna().mean()*100 #No NaN so far!

title          0.0
author         0.0
genre          0.0
rating         0.0
price          0.0
year           0.0
pages          0.0
reviews        0.0
amazon_year    0.0
dtype: float64

In [42]:
#Fill with languages and categories
# 
# 
import pandas as pd
import requests
import time

# Load your dataset
df = pd.read_csv("amazon_bestsellers_pages_filled.csv")

# Prepare new columns
df["Language"] = None
df["Categories"] = None

# Query Google Books API
def fetch_book_data(title, author):
    query = f"intitle:{title}+inauthor:{author}"
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if "items" in data and data["items"]:
            info = data["items"][0]["volumeInfo"]
        
            language = info.get("language")
            categories = ", ".join(info.get("categories", []))
            return pages, language, categories
    except:
        pass
    return None, None, None

# Fetch data row-by-row
for idx, row in df.iterrows():
    title, author = row["title"], row["author"]
    print(f"Fetching: {title} by {author}...")
    pages, language, categories = fetch_book_data(title, author)
    df.at[idx, "Total Pages"] = pages
    df.at[idx, "Language"] = language
    df.at[idx, "Categories"] = categories
    time.sleep(0.5)  # Prevent API rate limiting

# ------------------------------------------
# 📚 Override Categories based on genre/author, they were empty
# ------------------------------------------

# Art-related overrides
df.loc[df['genre'].str.contains("use of color", case=False, na=False), "Categories"] = "Art"
df.loc[df['author'].str.contains("Adult Coloring Book Designs", case=False, na=False), "Categories"] = "Art"
df.loc[df['genre'].str.contains("Coloring Books", case=False, na=False), "Categories"] = "Art"
df.loc[df['genre'].str.contains("Drawing", case=False, na=False), "Categories"] = "Art"


# Cooking-related overrides
df.loc[df['genre'].str.contains("Poultry", case=False, na=False), "Categories"] = "Cooking"

# Non Fiction
non_fiction_conditions = [
    "personal finance",
    "Social Psychology & Interactions",
    "Marriage & Divorce",
    "Reference",
    "School",
    "Parents",
    "Book Design",
    "Sounds",
    "Creativity",
    "Educational Psychology",
    "Quizzes",
    "Motherhood",
    "Faith",
    "Homeopathy",
    "Girls & Women"
    
]
for term in non_fiction_conditions:
    df.loc[df['genre'].str.contains(term, case=False, na=False), "Categories"] = "Non Fiction"

# Fiction
fiction_conditions = [
    "Epic",
    "Fantasy & Magic",
    "Mystery & Suspense",
    "Rabbits",
    "Southern",
    "Boys & Men",
    "Occult",
    "Paranormal",
    "Family Life"
]
for term in fiction_conditions:
    df.loc[df['genre'].str.contains(term, case=False, na=False), "Categories"] = "Fiction"
    df.loc[df['author'].str.contains("Rick Riordan", case=False, na=False), "Categories"] = " Juvenile Fiction"
    df.loc[df['author'].str.contains("J. K. Rowling", case=False, na=False), "Categories"] = " Fiction"
    df.loc[df['author'].str.contains("Ray Bradbury", case=False, na=False), "Categories"] = " Fiction"




# Save as new CSV
df.to_csv("amazon_bestsellers.csv", index=False)
print("✅ Enriched dataset saved as 'amazon_bestsellers.csv'")


Fetching: First 100 Words: A Padded Board Book by Roger Priddy...
Fetching: The Life-Changing Magic of Tidying Up: The Japanese Art of Decluttering and Organizing by Marie Kondō...
Fetching: Laugh-Out-Loud Jokes for Kids by Rob Elliott...
Fetching: Secret Garden: An Inky Treasure Hunt and Coloring Book for Adults by Johanna Basford...
Fetching: StrengthsFinder 2.0 by Gallup...
Fetching: Go Set a Watchman: A Novel by Harper Lee...
Fetching: Enchanted Forest: An Inky Quest and Coloring book (Activity Books, Mindfulness and Meditation, Illustrated Floral Prints) by Johanna Basford...
Fetching: All the Light We Cannot See by Anthony Doerr...
Fetching: The Girl on the Train by Paula Hawkins...
Fetching: Adult Coloring Book: Stress Relieving Patterns by Blue Star Coloring...
Fetching: Diary of a Wimpy Kid #10: Old School by Jeff Kinney...
Fetching: To Kill a Mockingbird by Harper Lee...
Fetching: Adult Coloring Book: Stress Relieving Animal Designs by Blue Star Coloring...
Fetching: Adult Co

In [44]:
#Make another column with genre, Binary column, fiction or non fiction
import pandas as pd
import requests
import time
df = pd.read_csv("amazon_bestsellers.csv")


# ----------------------------
# Create new "Genres" column
# ----------------------------
def classify_genre(cat):
    if not isinstance(cat, str):
        return "Non Fiction"
    
    cat_lower = cat.lower()
    
    if "nonfiction" in cat_lower or "non fiction" in cat_lower:
        return "Non Fiction"
    elif "fiction" in cat_lower:
        return "Fiction"
    else:
        return "Non Fiction"

df["Genres"] = df["Categories"].apply(classify_genre)

# Save the final CSV
df.to_csv("amazon_bestsellers_final.csv", index=False)
print("✅ File saved as 'amazon_bestsellers_final.csv'")


✅ File saved as 'amazon_bestsellers_final.csv'


In [46]:
#TRy 2
# 
import pandas as pd
import requests
import time
import re
import random
from tqdm import tqdm  # For a nicer progress bar

# Load your CSV
df = pd.read_csv("amazon_bestsellers_final.csv")

# Function to get publisher from Google Books API with improved search
def get_publisher_google_books(title, author=None):
    base_url = "https://www.googleapis.com/books/v1/volumes"
    
    # Clean the title for better search results
    clean_title = re.sub(r'[^\w\s]', '', title)
    
    # For long titles, use only the first part before any colon or parenthesis
    short_title = re.split(r'[:(]', clean_title)[0].strip()
    
    # Try different search strategies in order of specificity
    search_queries = []
    
    if author and pd.notna(author):
        # Full title with author
        search_queries.append((f"intitle:\"{clean_title}\" inauthor:{author}", 5))
        # Short title with author
        search_queries.append((f"intitle:\"{short_title}\" inauthor:{author}", 5))
    
    # Just the title queries
    search_queries.append((f"intitle:\"{clean_title}\"", 5))
    search_queries.append((f"intitle:\"{short_title}\"", 10))  # More results for shorter title
    
    # Try each search strategy until we find a publisher
    for query, max_results in search_queries:
        params = {"q": query, "maxResults": max_results}
        
        try:
            response = requests.get(base_url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            items = data.get("items", [])
            
            # Check through multiple results for better matching
            if items:
                for item in items:
                    volume_info = item.get("volumeInfo", {})
                    publisher = volume_info.get("publisher")
                    
                    if publisher:
                        return publisher
                
            # If we found items but no publisher, continue to next search strategy
        except Exception as e:
            print(f"Error fetching '{title}' with query '{query}': {e}")
        
        # Small delay between different search strategies
        time.sleep(0.5)
    
    return None

# Create a cache to store already fetched publishers
publisher_cache = {}

# Create a new column with publisher info
df["publisher"] = None  # Initialize the column

# Process in batches to handle API limits better
batch_size = 100
total_books = len(df)

for batch_start in range(0, total_books, batch_size):
    batch_end = min(batch_start + batch_size, total_books)
    batch = df.iloc[batch_start:batch_end]
    
    print(f"\nProcessing batch {batch_start//batch_size + 1}/{(total_books-1)//batch_size + 1}")
    
    for i, row in tqdm(batch.iterrows(), total=len(batch), desc="Fetching publishers"):
        title = row["title"]
        author = row.get("author", None)  # Use author info if available
        
        # Check cache first
        if title in publisher_cache:
            df.at[i, "publisher"] = publisher_cache[title]
            continue
            
        publisher = get_publisher_google_books(title, author)
        publisher_cache[title] = publisher  # Cache the result
        df.at[i, "publisher"] = publisher
        
        # Random delay between 1-2 seconds to prevent rate limiting
        time.sleep(1 + random.random())

# Manually fix known missing entries or common issues
manual_fixes = {
    "StrengthsFinder 2.0": "Gallup Press",
    "The Four Agreements": "Amber-Allen Publishing",
    "You Are a Badass": "Running Press",
    "The 5 Love Languages": "Northfield Publishing",
    "Crazy Rich Asians": "Doubleday",
    "The Alchemist": "HarperOne",
    "Enchanted Forest: An Inky Quest and Coloring book (Activity Books, Mindfulness and Meditation, Illustrated Floral Prints)": "Laurence King Publishing",
    "Ultimate Sticker Book: Frozen: More Than 60 Reusable Full-Color Stickers": "DK Children",
    "Where Is Baby's Belly Button? A Lift-the-Flap Book": "Little Simon",
    "Harry Potter and the Cursed Child, Parts 1 & 2, Special Rehearsal Edition Script": "Arthur A. Levine Books",
    "The Shack: Where Tragedy Confronts Eternity": "Windblown Media",
    "Last Week Tonight with John Oliver Presents: A Day in the Life of Marlon Bundo (HBO)": "Chronicle Books",
    "D&D 2014 Player's Handbook (Dungeons & Dragons Core Rulebook)": "Wizards of the Coast",
    "Crazy Rich Asians (Crazy Rich Asians Trilogy)": "Anchor Books",
    "Difficult Riddles For Smart Kids: 300 Difficult Riddles And Brain Teasers Families Will Love (Thinking Books for Kids)": "M & M Kidz",
    "Fahrenheit" : " Simon and Schuster",
    "Diary of a Wimpy Kid #13: Meltdown": "Harry N. Abrams",
    "Things We Never Got Over (Knockemout)": "Hodder & Stoughton",
    "Run, Rose, Run: A Novel": "Little, Brown and Company",
    "Haunting Adeline (Cat and Mouse Duet)" : "Independently published",
    "Murdle: Volume 1 (Murdle, 1)":"Griffin"

}

# Apply manual fixes
df["publisher"] = df.apply(
    lambda row: manual_fixes.get(row["title"], row["publisher"]), axis=1
)

# Count and report missing publishers
missing_count = df["publisher"].isna().sum()
if missing_count > 0:
    print(f"\n⚠️ {missing_count} books still missing publisher information.")
    print("Missing publisher for:")
    for title in df[df["publisher"].isna()]["title"].head(10):  # Show first 10 missing
        print(f"- {title}")
    if missing_count > 10:
        print(f"...and {missing_count - 10} more.")

# Save the updated dataset
output_file = "amazon_final.csv"
df.to_csv(output_file, index=False)
print(f"\n✅ Done! Saved to '{output_file}'")

# Print publisher statistics
publisher_counts = df["publisher"].value_counts().head(10)
print("\n📊 Top 10 publishers in the dataset:")
for publisher, count in publisher_counts.items():
    if pd.notna(publisher):
        print(f"- {publisher}: {count} books")


Processing batch 1/7


Fetching publishers: 100%|██████████| 100/100 [03:11<00:00,  1.91s/it]



Processing batch 2/7


Fetching publishers: 100%|██████████| 100/100 [02:31<00:00,  1.52s/it]



Processing batch 3/7


...


Processing batch 6/7


Fetching publishers: 100%|██████████| 100/100 [02:48<00:00,  1.69s/it]



Processing batch 7/7


Fetching publishers: 100%|██████████| 56/56 [01:15<00:00,  1.34s/it]


✅ Done! Saved to 'amazon_final.csv'

📊 Top 10 publishers in the dataset:
- Simon and Schuster: 55 books
- Random House: 29 books
- HarperCollins: 26 books
- Createspace Independent Publishing Platform: 23 books
- Penguin: 21 books
- Random House Books for Young Readers: 21 books
- National Geographic Books: 20 books
- Sourcebooks, Inc.: 19 books
- Penguin UK: 13 books
- Bloomsbury Publishing: 13 books


In [47]:
df = pd.read_csv("amazon_final.csv")

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        656 non-null    object 
 1   author       656 non-null    object 
 2   genre        656 non-null    object 
 3   rating       656 non-null    float64
 4   price        656 non-null    object 
 5   year         656 non-null    int64  
 6   pages        656 non-null    object 
 7   reviews      656 non-null    object 
 8   amazon_year  656 non-null    int64  
 9   Language     654 non-null    object 
 10  Categories   656 non-null    object 
 11  Total Pages  0 non-null      float64
 12  Genres       656 non-null    object 
 13  publisher    656 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 71.9+ KB


In [51]:
df.head(10)

,title,author,genre,rating,price,year,pages,reviews,amazon_year,Language,Categories,Total Pages,Genres,publisher
0,First 100 Words: A Padded Board Book,Roger Priddy,Vocabulary & Spelling,4.7,3..88,2011,24,"38,701 ratings",2015,en,Juvenile Nonfiction,NaN,Non Fiction,Priddy Books US
1,The Life-Changing Magic of Tidying Up: The Jap...,Marie Kondō,"Cleaning, Caretaking & Relocating",4.5,9..42,2017,224,"59,251 ratings",2015,en,House & Home,NaN,Non Fiction,Ten Speed Press
2,Laugh-Out-Loud Jokes for Kids,Rob Elliott,Humorous,4.6,4..64,2022,128,"12,125 ratings",2015,en,Juvenile Nonfiction,NaN,Non Fiction,HarperCollins
3,Secret Garden: An Inky Treasure Hunt and Color...,Johanna Basford,Use of Color,4.7,10..18,2013,96,"15,936 ratings",2015,en,Art,NaN,Non Fiction,National Geographic Books
4,StrengthsFinder 2.0,Gallup,Management,4.1,28..52,2007,175,"8,423 ratings",2015,es,Business & Economics,NaN,Non Fiction,Gallup Press
5,Go Set a Watchman: A Novel,Harper Lee,Cultural Heritage,3.8,12..95,2018,288,"54,069 ratings",2015,en,Fiction,NaN,Fiction,Harper
6,Enchanted Forest: An Inky Quest and Coloring b...,Johanna Basford,Coloring Books,4.8,14..19,2015,84,"8,419 ratings",2015,en,Art,NaN,Non Fiction,Laurence King Publishing
7,All the Light We Cannot See,Anthony Doerr,World War II & Holocaust,4.5,12..93,2014,531,"232,435 ratings",2015,en,Fiction,NaN,Fiction,Simon and Schuster
8,The Girl on the Train,Paula Hawkins,Suspense,4.1,12..20,2014,336,"494,411 ratings",2015,en,Alcoholics,NaN,Non Fiction,Random House
9,Adult Coloring Book: Stress Relieving Patterns,Blue Star Coloring,Use of Color,4.5,9..95,2015,96,"4,363 ratings",2015,en,Art,NaN,Non Fiction,Blue Star Press


In [50]:
df.isna().mean()*100

title            0.000000
author           0.000000
genre            0.000000
rating           0.000000
price            0.000000
year             0.000000
pages            0.000000
reviews          0.000000
amazon_year      0.000000
Language         0.304878
Categories       0.000000
Total Pages    100.000000
Genres           0.000000
publisher        0.000000
dtype: float64

In [53]:
df.drop(axis = 1, columns = ["Total Pages", "Language"] )

,title,author,genre,rating,price,year,pages,reviews,amazon_year,Categories,Genres,publisher
0,First 100 Words: A Padded Board Book,Roger Priddy,Vocabulary & Spelling,4.7,3..88,2011,24,"38,701 ratings",2015,Juvenile Nonfiction,Non Fiction,Priddy Books US
1,The Life-Changing Magic of Tidying Up: The Jap...,Marie Kondō,"Cleaning, Caretaking & Relocating",4.5,9..42,2017,224,"59,251 ratings",2015,House & Home,Non Fiction,Ten Speed Press
2,Laugh-Out-Loud Jokes for Kids,Rob Elliott,Humorous,4.6,4..64,2022,128,"12,125 ratings",2015,Juvenile Nonfiction,Non Fiction,HarperCollins
3,Secret Garden: An Inky Treasure Hunt and Color...,Johanna Basford,Use of Color,4.7,10..18,2013,96,"15,936 ratings",2015,Art,Non Fiction,National Geographic Books
4,StrengthsFinder 2.0,Gallup,Management,4.1,28..52,2007,175,"8,423 ratings",2015,Business & Economics,Non Fiction,Gallup Press
...,...,...,...,...,...,...,...,...,...,...,...,...
651,A Court of Thorns and Roses (A Court of Thorns...,Sarah J. Maas,Paranormal,4.6,10..05,2024,448,"279,098 ratings",2025,Fiction,Fiction,Bloomsbury Publishing
652,Fuzzy Hygge: Cute and Cozy Coloring Book for A...,Vivi Tinta,Animals,4.8,7..99,2024,88,"4,036 ratings",2025,Art,Non Fiction,Amelia Green
653,"Oh, the Places You'll Go!",Dr. Seuss,Classics,4.9,8..48,1990,56,"52,232 ratings",2025,Juvenile Fiction,Fiction,Random House Books for Young Readers
654,The Boyfriend,Freida McFadden,Murder,4.3,9..99,1990,368,"191,449 ratings",2025,Fiction,Fiction,Poisoned Pen Press
